In [ ]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
p = 0.01

df = pd.read_csv(
    'MICRODADOS_ENEM_2019.csv',
    header=0,
    encoding = "ISO-8859-1",
    skiprows=lambda i: i > 0 and random.random() > p,
    sep=';'
)

df.head()

In [ ]:
df.columns

# Funções 

In [ ]:
def subgrupo(x):
    if x =='A':
        return 0
    if any((x =='B',x=='C',x=='D',x=='E')):
        return 1
    if any((x=='F',x=='G',x=='H',x=='I')):
        return 2
    if any((x=='J',x=='K',x=='L',x=='M')):
        return 3
    if any((x=='N',x=='O',x=='P',x=='Q')):
        return 4
    
df['Subgrupo_Renda'] = df['Q006'].apply(subgrupo)


def celular(x):
    if x =='A':
        return 'Sem Celular'
    if x == 'B' or x =='C' or  x =='D' or  x =='E':
        return 'Com Celular'

df['Celular'] = df['Q022'].apply(celular)


def computador(x):
    if x == 'A':
        return 'Sem Computador'
    else:
        return 'Com Computador'

df['Computador'] = df['Q024'].apply(computador)


def internet (x):
    if x =='A':
        return 'Sem Internet'
    else:
        return 'Com Internet'

df['Internet'] = df['Q025'].apply(internet)


In [ ]:
# 	Nenhuma renda. 0
# Vou comparar com o Q

# B	Até R$ 998,00
# C	De R$ 998,01 até R$ 1.497,00.
# D	De R$ 1.497,01 até R$ 1.996,00.
# E	De R$ 1.996,01 até R$ 2.495,00.

# F	De R$ 2.495,01 até R$ 2.994,00.
# G	De R$ 2.994,01 até R$ 3.992,00.
# H	De R$ 3.992,01 até R$ 4.990,00.
# I	De R$ 4.990,01 até R$ 5.988,00.

# J	De R$ 5.988,01 até R$ 6.986,00.
# K	De R$ 6.986,01 até R$ 7.984,00.
# L	De R$ 7.984,01 até R$ 8.982,00.
# M	De R$ 8.982,01 até R$ 9.980,00.

# N	De R$ 9.980,01 até R$ 11.976,00.
# O	De R$ 11.976,01 até R$ 14.970,00.
# P	De R$ 14.970,01 até R$ 19.960,00.
# Q	Mais de R$ 19.960,00.


#Dropei valores nulos, assim como a média, notas (pessoas que n compareceram, ou zeraram a redação)
df = df.dropna()

#media
media = (df.NU_NOTA_CN+df.NU_NOTA_CH+df.NU_NOTA_LC+df.NU_NOTA_MT+df.NU_NOTA_REDACAO)/5
df['media'] = media


df.head(100)

### GRÁFICO COMPARANDO OS SUBGRUPOS, ANALISANDO A RENDA MENSAL


### Subgrupo 0
 Nenhuma renda

### Subgrupo 1
998,00 até 2495,00

### Subgrupo 2
2495,01 até 5988,00

### Subgrupo 3
5998,01 até 6986,00

### Subgrupo 4
Mais de 6986,00

Valores em real



In [ ]:
fig, ax = plt.subplots( figsize=(20, 10)) #cria uam figura com 1 linha, 2 colunas, de tamanho 20x10

sns.set_style('dark')
sns.set_context('talk')
sns.kdeplot(
   data=df, x="media", hue="Subgrupo_Renda",
   fill=True, common_norm=False, palette="crest",
   alpha=.5, linewidth=0,
)
plt.show()

### GRAFICO COMPARANDO PESSOAS COM OU SEM CELULAR (INTERNET INCLUSA)

In [ ]:
fig, ax = plt.subplots( figsize=(20, 10)) #cria uam figura com 1 linha, 2 colunas, de tamanho 20x10
cel_internet = df.loc[df['Q025'] == 'B']
cel_internet

sns.set_style('dark')
sns.set_context('talk')

sns.kdeplot(
   data=cel_internet, x="media", hue="Celular",
   fill=True, common_norm=False, palette="magma",
   alpha=.5, linewidth=0,
)
plt.show()

### GRAFICO COMPARANDO PESSOAS COM OU SEM COMPUTADOR (INTERNET INCLUSA)

In [ ]:
fig, ax = plt.subplots( figsize=(20, 10)) #cria uam figura com 1 linha, 2 colunas, de tamanho 20x10
pc_internet = df.loc[df['Q025'] == 'B']
pc_internet

sns.set_style('dark')
sns.set_context('talk')

sns.kdeplot(
   data=pc_internet, x="media", hue="Computador",
   fill=True, common_norm=False, palette="viridis",
   alpha=.5, linewidth=0,
)
plt.show()

In [ ]:

sns.set_style('dark')
sns.set_context('talk')
sns.displot(
   data= pc_internet,x="media", hue="Computador",
   fill=True, common_norm=False, palette="viridis",
   alpha=.5, linewidth=0,
    
)
plt.show()

# Gráfico pessoas com ou sem internet

In [ ]:
fig, ax = plt.subplots( figsize=(20, 10)) #cria uam figura com 1 linha, 2 colunas, de tamanho 20x10

sns.set_style('dark')
sns.set_context('talk')
sns.kdeplot(
   data= df,x="media", hue="Internet",
   fill=True, common_norm=False, palette="magma",
   alpha=.5, linewidth=0,
)
plt.show()

In [ ]:
sns.set_style('dark')
sns.set_context('talk')

sns.displot(
   data= df,x="media", hue="Internet",
   fill=True, common_norm=False, palette="magma",
   alpha=.5, linewidth=0,
     kde=False, bins= 20
)
plt.show()

In [ ]:
# NU_NOTA_REDACAO	Nota da prova de redação
# NU_NOTA_MT	Nota da prova de Matemática
fig, ax = plt.subplots(figsize=(20, 10))
sns.kdeplot(df.(media)['NU_NOTA_REDACAO'], ax=ax, shade=True, label='Female age') # plota a distribuiçao da coluna age entre as pessaos femininas
sns.kdeplot(df.media['NU_NOTA_MT'], ax=ax, shade=True, label='Male age') #plota a distribuiçao da coluna age entre as pessoas masculinas
ax.set_title('Distribuição das idades de acordo com o sexo') # coloca titulo
plt.show()

#df.query("Coluna == 'valor'") ==> seleciona somente as linhas cujo valor da coluna é igual ao valor desejado
